In [1]:
import pandas as pd
import regex as re
import pygsheets
from datetime import date, timedelta, datetime
import numpy as np
import geopandas as gpd

In [2]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
        if other_damages == True:
            NO_DAMAGES = len(re.findall('\|', SUB_DETAIL))
            print(NO_DAMAGES)
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES-1
            continue 
            
        NO_DAMAGES_DateAlgo = len(re.findall('2023',SUB_DETAIL)) + len(re.findall(r'\d+/\d+/\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+-\d+-\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+\.\d+\.\d{2}\s+', SUB_DETAIL))
        # Number of 2022 in the DETAIL +  Number of dd/mm/22 in the DETAIL +  Number of dd-mm-22 in the DETAIL + Number of dd.mm.22 in the DETAIL
        NO_DAMAGES_LongAlgo = len(re.findall('Long -',SUB_DETAIL)) 
    
        if float(ROW['Number of Damages (Long)'])==float(ROW['Number']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        elif float(ROW['Number of Damages (Long)'])>float(ROW['Number of Damages (Date)']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        else:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_DateAlgo
    
    
    return REV_CIRCLE_DAMAGES_DICT    

In [3]:
ASSAM_REVENUE_CIRCLES = pd.read_csv('Data/Assam_Maps/ASSAM_REVENUE_CIRCLES_FRIMS_NAMES.csv')
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-I\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-II\)','', regex=True)

ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.strip()

# Table of Contents:
* [Roads damaged per revenue circle](#roadsdamaged)
* [Embankment damages per revenue circle](#embankmentdamages)
* [Embankments affected per revenue circle](#embankmentaffected)
* [Bridges damaged per revenue circle](#bridgesdamaged)

## EMBANKMENT AFFECTED <a class="anchor" id="embankmentaffected"></a>

In [6]:
embankments_affected_df = pd.read_csv('Data_2023/Cleaned Data/DISTRICTS_FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv')
embankments_affected_df['Details'] = embankments_affected_df['Details'].str.replace('\n', ' ')


embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Baghbar','Baghbor',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dotma','Dotoma',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Palashbari','Palasbari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Naduar','Na-Duar',regex=True)

embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


embankments_affected_df.tail()

,Date,District,Number,Details,state,ID,assam_dist
46,2023-07-12,DHEMAJI,1,Jonai - Tako to Borang charabari under PMGSY P...,ASSAM,7,2011Dhem
47,2023-07-14,MAJULI,1,Ujani Majuli - UN-Embankment | Borpamua | Affe...,ASSAM,18,2019Maju
48,2023-07-15,GOLAGHAT,1,Bokakhat - Brahmaputra Dyke from KRF boundary ...,ASSAM,6,2011Gola
49,2023-07-15,MAJULI,2,Ujani Majuli - Un-Embankment | Jengrai Kumarba...,ASSAM,18,2019Maju
50,2023-07-17,BISWANATH,1,Gohpur - Bank erosion is observed on R/Bank em...,ASSAM,24,2019Bisw


In [7]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_affected_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_affected_df[embankments_affected_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

DHEMAJI
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
BISWANATH
['Biswanath -', 'Gohpur -', 'Halem -']
----
DARRANG
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
TAMULPUR
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
GOLAGHAT
['Bokakhat -', 'Dergaon -', 'Golaghat -', 'Khumtai -', 'Morangi -', 'Sarupathar -']
----
KARIMGANJ
['Badarpur -', 'Karimganj -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
GOALPARA
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
LAKHIMPUR
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
NALBARI
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
NAGAON
['Dhing -', 'Kaliabor -', 'Kampur -', 'Nagaon -', 'Raha -', 'Rupahi -', 'Samaguri -']
----
BAJALI
['Bajali -', 'Jalah -', 'Sarupeta -']
-

/tmp/ipykernel_11725/797364889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [8]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [9]:
embankment_affected_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_affected_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_affected_revcircle_df.loc[i,'District'] = row['District']
        embankment_affected_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_affected_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_affected_revcircle_df.Date = pd.to_datetime(embankment_affected_revcircle_df.Date)

In [10]:
embankment_affected_revcircle_df.RevenueCircle =embankment_affected_revcircle_df.RevenueCircle.str.strip()
embankment_affected_revcircle_df.loc[embankment_affected_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [11]:
embankment_affected_revcircle_df = pd.merge(embankment_affected_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [12]:
embankment_affected_revcircle_df = embankment_affected_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [13]:
embankment_affected_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv', index=False)

## EMBANKMENT BREACHES <a class="anchor" id="embankmentbreaches"></a>

In [17]:
embankments_breached_df = pd.read_csv('Data_2023/Cleaned Data/DISTRICTS_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')
embankments_breached_df['Details'] = embankments_breached_df['Details'].str.replace('\n', ' ')


embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Baghbar','Baghbor',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Dotma','Dotoma',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Palashbari','Palasbari',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Naduar','Na-Duar',regex=True)


embankments_breached_df['District'] = embankments_breached_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
embankments_breached_df['District'] = embankments_breached_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


embankments_breached_df.tail()

,Date,District,Number,Details,state,ID,assam_dist
25,2023-06-28,BISWANATH,1,Halem - At upstream ofTinikuniaon R/Bank TeaGa...,ASSAM,24,2019Bisw
26,2023-06-29,JORHAT,1,Teok - Teok river left ring bund | Dakhin Duli...,ASSAM,22,2011Jorh
27,2023-07-09,JORHAT,1,Teok - Right ring bund of river Teok | Dakhin ...,ASSAM,22,2011Jorh
28,2023-07-10,BISWANATH,1,Halem - Tupung river embankment | Jogibari | S...,ASSAM,24,2019Bisw
29,2023-07-11,UDALGURI,1,Udalguri - Dhansiri Irrigation Project | Vilag...,ASSAM,21,2011Udal


In [18]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_breached_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_breached_df[embankments_breached_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

BISWANATH
['Biswanath -', 'Gohpur -', 'Halem -']
----
LAKHIMPUR
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
DARRANG
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
GOALPARA
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
UDALGURI
['Dalgaon -', 'Dhekiajuli -', 'Harisinga -', 'Kalaigaon -', 'Khoirabari -', 'Mangaldoi -', 'Mazbat -', 'Pathorighat -', 'Udalguri -']
----
SONITPUR
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
NAGAON
['Dhing -', 'Kaliabor -', 'Kampur -', 'Nagaon -', 'Raha -', 'Rupahi -', 'Samaguri -']
----
BAKSA
['Baganpara -', 'Barama -', 'Barnagar -', 'Baska -', 'Ghograpar -', 'Jalah -', 'Sarupeta -']
----
NALBARI
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
BAJALI
['Bajali -', 'Jalah -', 'Sarupeta -']
----
JORHAT
['Jorhat East -', 'Jorhat West -', 'Marian

/tmp/ipykernel_11725/1701030496.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [19]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [20]:
embankment_breaches_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_breaches_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_breaches_revcircle_df.loc[i,'District'] = row['District']
        embankment_breaches_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_breaches_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_breaches_revcircle_df.Date = pd.to_datetime(embankment_breaches_revcircle_df.Date)

In [21]:
embankment_breaches_revcircle_df.RevenueCircle =embankment_breaches_revcircle_df.RevenueCircle.str.strip()
embankment_breaches_revcircle_df.loc[embankment_breaches_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [22]:
embankment_breaches_revcircle_df = pd.merge(embankment_breaches_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [23]:
embankment_breaches_revcircle_df = embankment_breaches_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [24]:
embankment_breaches_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv', index=False)

## BRIDGES DAMAGED <a class="anchor" id="bridgesdamaged"></a>

In [25]:
bridges_df = pd.read_csv('Data_2023/Cleaned Data/DISTRICTS_FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv')
bridges_df['Details'] = bridges_df['Details'].str.replace('\n', ' ')


bridges_df['Details'] = bridges_df.Details.str.replace('Baghbar','Baghbor',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dotma','Dotoma',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Palashbari','Palasbari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Naduar','Na-Duar',regex=True)


bridges_df['District'] = bridges_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
bridges_df['District'] = bridges_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


bridges_df.tail()

,Date,District,Number,Details,state,ID,assam_dist
27,2023-07-10,BISWANATH,1,Halem - R.C.C Br.No.1/2 - Approch | Borjohabar...,ASSAM,24,2019Bisw
28,2023-07-11,GOLAGHAT,2,"Bokakhat - Kamalamiri Ali, L=6.00km ( RCC Br. ...",ASSAM,6,2011Gola
29,2023-07-13,BARPETA,1,Sarthebari - Baithabhanga Pithadi Road SPT Br ...,ASSAM,32,2011Barp
30,2023-07-17,BONGAIGAON,1,Manikpur - Kawadi to Sonaikhola Road | Sonaikh...,ASSAM,10,2011Bong
31,2023-07-18,DHEMAJI,1,Dhemaji - Shilpata road from Tanganapara to Ma...,ASSAM,7,2011Dhem


In [26]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in bridges_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = bridges_df[bridges_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

DHEMAJI
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
SONITPUR
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
DARRANG
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
KOKRAJHAR
['Bagribari -', 'Bhawraguri -', 'Chapar -', 'Dhubri -', 'Dotoma -', 'Golokganj -', 'Gossaigaon -', 'Kokrajhar -']
----
BARPETA
['Baghbor -', 'Barnagar -', 'Barpeta -', 'Chenga -', 'Kalgachia -', 'Sarthebari -']
----
BONGAIGAON
['Boitamari -', 'Bongaigaon -', 'Manikpur -', 'Sidli -', 'Srijangram -']
----
BAKSA
['Baganpara -', 'Barama -', 'Barnagar -', 'Baska -', 'Ghograpar -', 'Jalah -', 'Sarupeta -']
----
NALBARI
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
BISWANATH
['Biswanath -', 'Gohpur -', 'Halem -']
----
DHUBRI
['Agamoni -', 'Bagribari -', 'Bilasipara -', 'Chapar -', 'Dhubri -', 'Golakganj -', 'Gossaigaon -']
----
BAJALI
['Bajali -', 'J

/tmp/ipykernel_11725/1577161483.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [27]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [28]:
bridges_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        bridges_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        bridges_revcircle_df.loc[i,'District'] = row['District']
        bridges_revcircle_df.loc[i,'Number of damages'] = no_damages
        bridges_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
bridges_revcircle_df.Date = pd.to_datetime(bridges_revcircle_df.Date)

In [29]:
bridges_revcircle_df.RevenueCircle =bridges_revcircle_df.RevenueCircle.str.strip()
bridges_revcircle_df.loc[bridges_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [30]:
bridges_revcircle_df = pd.merge(bridges_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [31]:
bridges_revcircle_df = bridges_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [32]:
bridges_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv', index=False)

## ROADS DAMAGED <a class="anchor" id="roadsdamaged"></a>

In [33]:
roads_damaged_df = pd.read_csv('Data_2023/Cleaned Data/DISTRICTS_FRIMS_ROADS_DAMAGED_MASTER_2023.csv')
roads_damaged_df['Details'] = roads_damaged_df['Details'].str.replace('\n', ' ')

roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Baghbar','Baghbor',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dotma','Dotoma',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Palashbari','Palasbari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Naduar','Na-Duar',regex=True)


roads_damaged_df['District'] = roads_damaged_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
roads_damaged_df['District'] = roads_damaged_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


roads_damaged_df.head()

,Date,District,Number,Details,state,ID,assam_dist
0,2023-06-11,BISWANATH,1,Gohpur - Singarajan to Sutardoloni Kutcha Road...,ASSAM,24,2019Bisw
1,2023-06-12,DARRANG,1,Mangaldoi - Approach road Noanadi Kachia Bund ...,ASSAM,26,2011Darr
2,2023-06-12,BISWANATH,6,Gohpur - Road from Borigaon to Deonabori. (A c...,ASSAM,24,2019Bisw
3,2023-06-13,LAKHIMPUR,1,Narayanpur - Balikuchi Bishnupur Road at Ch 35...,ASSAM,15,2011Lakh
4,2023-06-14,LAKHIMPUR,2,Nowboicha - Dolohat to Laluk Kachajuli PWD Roa...,ASSAM,15,2011Lakh


In [34]:
roads_damaged_df = roads_damaged_df.sort_values(by='Date')

In [35]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in roads_damaged_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = roads_damaged_df[roads_damaged_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

BISWANATH
['Biswanath -', 'Gohpur -', 'Halem -']
----
DARRANG
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
LAKHIMPUR
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
DHEMAJI
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
DIMA HASAO
['Haflong -', 'Mahur -', 'Maibong -', 'Umrangso -']
----
GOALPARA
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
NALBARI
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
UDALGURI
['Dalgaon -', 'Dhekiajuli -', 'Harisinga -', 'Kalaigaon -', 'Khoirabari -', 'Mangaldoi -', 'Mazbat -', 'Pathorighat -', 'Udalguri -']
----
TAMULPUR
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
KAMRUP
['Azara -', 'Boko -', 'Chamaria -', 'Chhaygaon -', 'Goroimari -', 'Hajo -', 'Jalah -', 'Kamalpur -', 'Kayan -',

/tmp/ipykernel_11725/993252146.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


SONITPUR
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
CHARAIDEO
['Mahmora -', 'Sapekhati -', 'Sonari -']
----
MORIGAON
['Bhuragaon -', 'Laharighat -', 'Mayong -', 'Mikirbheta -', 'Morigaon -']
----
DIBRUGARH
['Chabua -', 'Dibrugarh East -', 'Dibrugarh West -', 'Moran -', 'Naharkatia -', 'Tengakhat -', 'Tingkhong -']
----


In [36]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [37]:
road_damages_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        road_damages_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        road_damages_revcircle_df.loc[i,'District'] = row['District']
        road_damages_revcircle_df.loc[i,'Number of damages'] = no_damages
        road_damages_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
road_damages_revcircle_df.Date = pd.to_datetime(road_damages_revcircle_df.Date)

In [38]:
road_damages_revcircle_df.RevenueCircle =road_damages_revcircle_df.RevenueCircle.str.strip()
road_damages_revcircle_df.loc[road_damages_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [39]:
road_damages_revcircle_df = pd.merge(road_damages_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [40]:
road_damages_revcircle_df = road_damages_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [41]:
road_damages_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_ROADS_DAMAGED_MASTER_2023.csv', index=False)

# Lat-Lons

In [77]:
def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

In [78]:
df = pd.read_csv('Data_2023/Cleaned Data/DISTRICTS_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')
df.tail()

,Date,District,Number,Details,state,ID,assam_dist
25,2023-06-28,BISWANATH,1,Halem - At upstream ofTinikuniaon R/Bank TeaGa...,ASSAM,24,2019Bisw
26,2023-06-29,JORHAT,1,Teok - Teok river left ring bund | Dakhin Duli...,ASSAM,22,2011Jorh
27,2023-07-09,JORHAT,1,Teok - Right ring bund of river Teok | Dakhin ...,ASSAM,22,2011Jorh
28,2023-07-10,BISWANATH,1,Halem - Tupung river embankment | Jogibari | S...,ASSAM,24,2019Bisw
29,2023-07-11,UDALGURI,1,Udalguri - Dhansiri Irrigation Project | Vilag...,ASSAM,21,2011Udal


In [85]:

dates_list = []
locs_list = []

for idx, row in df.iterrows():
    details = row['Details']
    long_lats = re.findall('\d{1,2}\.\d{6}', details)
    locs_list.append(long_lats)
    #long_lats = [x for x in long_lats if float(x) != 0]
    repeat = int(len(long_lats)/2)
    dates = [row['Date']]*repeat
    dates_list.append(dates)

In [98]:
longitudes = []
latitudes = []

for loc in flatten_list(locs_list):
    if float(loc) < 80:
        latitudes.append(loc)
    else:
        longitudes.append(loc)

In [99]:
len(longitudes)

46

In [100]:
len(latitudes)

56

In [95]:
len(flatten_list(dates_list))

51

In [97]:
len(flatten_list(locs_list))

102

In [56]:
breaches_lat_lon_df = pd.DataFrame([dates_list, longitudes,latitudes]).T
breaches_lat_lon_df.columns = ['Date', 'Longitude', 'Latitude']
breaches_lat_lon_df.to_csv('Data_2023/Cleaned Data/LAT_LONS_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')

In [58]:
breaches_lat_lon_df

,Date,Longitude,Latitude
0,"[2023-06-11, 2023-06-11]",93.628098,26.886123
1,"[2023-06-12, 2023-06-12]",93.555650,27.095780
2,"[2023-06-12, 2023-06-12, 2023-06-12, 2023-06-1...",91.992661,26.440664
3,"[2023-06-12, 2023-06-12]",91.960690,26.442980
4,"[2023-06-14, 2023-06-14, 2023-06-14, 2023-06-14]",91.960690,26.442980
5,"[2023-06-15, 2023-06-15]",93.691218,26.900444
6,"[2023-06-16, 2023-06-16, 2023-06-16, 2023-06-16]",94.006159,17.156999
7,"[2023-06-16, 2023-06-16, 2023-06-16, 2023-06-16]",90.753727,26.020408
8,"[2023-06-17, 2023-06-17]",91.776100,26.711700
9,"[2023-06-17, 2023-06-17]",91.986521,26.562401
